In [ ]:
%pip install --upgrade pip -q

In [ ]:
%pip install python-dotenv -qU
%pip install 'autogen-agentchat==0.4.0.dev8' -qU
%pip install 'autogen-ext[openai,azure]==0.4.0.dev8' -qU

In [ ]:
import autogen_agentchat as chat

chat.__version__

In [6]:
import os
from dotenv import load_dotenv

load_dotenv("../../../.env")
os.environ.get("AZURE_OPENAI_ENDPOINT_GPT_4o")

api_key = os.environ["AZURE_OPENAI_API_KEY_GPT_4o"]

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient


# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


async def main() -> None:
    # Define an agent
    weather_agent = AssistantAgent(
        name="weather_agent",
        model_client=AzureOpenAIChatCompletionClient(
            azure_endpoint='https://xulei-omni.openai.azure.com',
            model="gpt-4o",
            azure_deployment="gpt-4o-lei",
            api_key=api_key,
            api_version="2024-06-01"
        ),
        tools=[get_weather],
    )

    # Define termination condition
    termination = TextMentionTermination("TERMINATE")

    # Define a team
    agent_team = RoundRobinGroupChat(
        [weather_agent], termination_condition=termination)

    # Run the team and stream messages to the console
    stream = agent_team.run_stream(task="What is the weather in New York?")
    await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

### Summary
The code snippet above introduces two high level concepts in AgentChat: Agent and Team. An Agent helps us define what actions are taken when a message is received. Specifically, we use the AssistantAgent preset - an agent that can be given access to a model (e.g., LLM) and tools (functions) that it can then use to address tasks. A Team helps us define the rules for how agents interact with each other. In the RoundRobinGroupChat team, agents respond in a sequential round-robin fashion. In this case, we have a single agent, so the same agent is used for each round.